In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
import requests
import time
import re
import pandas as pd

In [ ]:
start_time = time.time()

path = 'c:/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(path)

df = pd.DataFrame()

#for url in urlList:
driver.get(url)
html = driver.page_source
soup = bs(html, 'lxml')
    
item = {}  
    
item['manufacturer'] = soup.select_one('div.goods_info > div:nth-of-type(1) > span:nth-of-type(1) > em').text # 제조사
item['brand'] =  soup.select_one('div.goods_info > div:nth-of-type(1) > span:nth-of-type(2) > em').text # 브랜드
item['heart'] = soup.select_oner('#jjim > em.cnt._keepCount').text # 찜하기 수 

# 평점별 리뷰개수
p = re.compile(r'(?!=[(])\d+(?=[)])')
item['total_review'] = int(p.search(soup.select_one('#_score_filter > li:nth-of-type(1)').get_text()).group())
item['review_score_5'] = int(p.search(soup.select_one('#_score_filter > li:nth-of-type(2)').get_text()).group())
item['review_score_4'] = int(p.search(soup.select_one('#_score_filter > li:nth-of-type(3)').get_text()).group())
item['review_score_3'] = int(p.search(soup.select_one('#_score_filter > li:nth-of-type(4)').get_text()).group())
item['review_score_2'] = int(p.search(soup.select_one('#_score_filter > li:nth-of-type(5)').get_text()).group())
item['review_score_1'] = int(p.search(soup.select_one('#_score_filter > li:nth-of-type(6)').get_text()).group())

page = soup.select_one('#_review_paging > a.next_end')
if page:
    p = re.compile(r'(?!=page.)\d+(?=,)')
    lastPage = int(p.search(str(page)).group()) # 총 페이지 개수
else:
    lastPage = int(soup.select('#_review_paging > a')[-1].text) # 다음 페이지 버튼 없는 경우

name = []
path = []
star = []
date = []
subject = []   
content = []
img = []

#driver.find_element_by_css_selector('#_review_sort_select > span:nth-child(2) > a').click() # 리뷰 최신순

page = 1
while page<=lastPage: #보고 싶은 페이지 수 만큼

    html = driver.page_source
    soup = bs(html,'lxml')
    
    name.extend([w.get_text() for w in soup.select('div.avg_area > span.info > span.name')]) # 작성자 
    path.extend([w.get_text() for w in soup.select('div.avg_area > span.info > span.path')]) # 구매처
    star.extend([w.get_text() for w in soup.select('div.avg_area > a > span.curr_avg')[2:]]) # 별점
    date.extend([w.get_text() for w in soup.select('div.avg_area > span.date')]) # 작성일
    subject.extend([w.get_text() for w in soup.find_all(class_='subjcet')]) # 제목 텍스트
    content.extend([w.get_text().replace('\xa0', '') for w in soup.find_all(class_='atc')]) # 본문 텍스트
        
    for s in soup.find_all(class_='atc_area'): # 이미지 유무
        if 'img_box' in str(s):
            img.append('Y')
        else:
            img.append('N')            
    
    #page.click() # error -> 클릭할 버튼을 specific하게 지정 필요
    if page < lastPage:
        if lastPage > 10:
            driver.find_elements_by_xpath('//*[@id="_review_paging"]/a')[:-1][page-1].click()
        #pageList = driver.find_elements_by_xpath('//*[@id="_review_paging"]/a')
        else:
            driver.find_elements_by_xpath('//*[@id="_review_paging"]/a')[page-1].click() # 다음 페이지
    page+=1    

item['name'] = name
item['path'] = path
item['star'] = star
item['date'] = date
item['subject'] = subject
item['content'] = content
item['image'] = img  

df = pd.concat([df, pd.DataFrame(item)])

#urlList for문 끝
df['date'] = pd.to_datetime(df['date'])


print("--- %.4f seconds ---" % (time.time() - start_time))

In [ ]:
df.drop_duplicates().shape[0] == int(soup.select_one('#fixed_tab_area > div > ul > li.mall_review.on > a > em').text) # 리뷰수 일치하는지